In this post, I explain how find the ensemble average of classification models using the [softmax](https://en.wikipedia.org/wiki/Softmax_function) function.

## Ensemble average of logistic models

For linear regression, the predicted outcome $y(w,x)$ for an observed set of features $x$ is
\begin{align}
y(w,x)
 &= w_0 + w_1 x_1 + \cdots + w_n x_n
\end{align}
where $w_{i}$ is the learned weights of the linear model. In the case of logistic regression used for binary outcomes, the predicted outcome is limited to be between $0$ and $1$. Typically, the sigmoid function $\sigma(z)$ is used
\begin{align}
y(w,x)
 &= \sigma(z) \\
 &= \frac{1}{1+e^{-z(w,x)}} \\
z(w,x)
 &= w_0 + w_1 x_1 + \cdots + w_n x_n.
\end{align}

For multiple models that use the logistic function as the outcome, the ensemble average can be calculated by first taking the mean value of the function $z$ and then inputing the result into the logistic function
\begin{align}
\bar z(w,x)
 &= \frac{1}{N}\sum_i z_i \\
 &= \frac{1}{N}\left[\sum_{i} w_{i0} + w_{i1} x_1 + \cdots + w_{in} x_n\right] \\
\bar y(w,x)
 &= \sigma(\bar z)
\end{align}
where $N$ is the number of models in the ensemble.

Many machine learning codes only give you the predicted outcome of each model. When this is the case, the logistic function can be inverted to obtain the $z_i$s. The inverse logistic function is the logit function
\begin{align}
\mathrm{logit}(y)
 &= \ln\frac{y}{1 - y}
\end{align}
so the ensemble mean is
\begin{align}
\bar y = \sigma\left(\frac{1}{N}\sum_i \mathrm{logit}(y_i) \right).
\end{align}

## Derivation for softmax ensemble average

The softmax function is a generalization of the sigmiod function for when there are multiple outcomes. It is defined as
\begin{align}
y(z_i) &= \frac{e^{z_i}}{\sum_{k}e^{z_k}}
\end{align}
and it reduced to the sigmoid function when there are only two outcomes.

Inverting the softmax function in order to take the mean follows the same steps as inverting the sigmoid function. The only difference is now the denominator is more complicated. When there are multiple models to choose from, I will define a common softmax function for all of them
\begin{align}
y(z_{ij}) &= \frac{e^{z_{ij}}}{\sum_{k}e^{z_{kj}}}
\end{align}
where the index $j$ identifies which model is used. The denominator is a constant for each model, so I will replace it with $f_j$. Additionally, I will take the log of the equation and solve for $z_{ij}$
\begin{align}
y_{ij}
 &= e^{z_{ij}}f_j \\
\ln y_{ij}
 &= z_{ij} + \ln f_j \\
z_{ij} &= \ln y_{ij} - \ln f_j.
\end{align}
These are the terms I want the average over all models.
\begin{align}
\bar z_i
 &= \frac{1}{N}\sum_j z_{ij} \\
 &= \frac{1}{N}\sum_j \ln y_{ij} - \frac{1}{N}\sum_j \ln f_j.
\end{align}
Finally, the softmax of the average $z_i$'s is
\begin{align}
softmax(\bar z_i)
&= \frac{e^{\frac{1}{N}\sum_j \ln y_{ij} - \frac{1}{N}\sum_j \ln f_j}}
{\sum_k e^{\frac{1}{N}\sum_j \ln y_{kj} - \frac{1}{N}\sum_j \ln f_j}} \\
&= \frac{e^{\frac{1}{N}\sum_j \ln y_{ij}}}
{\sum_k e^{\frac{1}{N}\sum_j \ln y_{kj}}} \\
&= \frac{e^{\overline{\ln y_{ij}}}}
{\sum_k e^{\overline{\ln y_{kj}}}} \\
\end{align}
Some examples are next.

## Examples
First, I define a simple softmax function using numpy.

In [1]:
from __future__ import print_function, division
import numpy as np

np.set_printoptions(precision = 4)

def softmax(z):
    return (np.exp(z).T / np.exp(z).T.sum(axis = 0)).T

# Higher values produce higher probability.
z = [3.14,0.2,10.]
print(softmax(z))

# All probabilities add up to 1
assert np.isclose(softmax(z).sum(), 1.)

[  1.0478e-03   5.5390e-05   9.9890e-01]


For a typicaly classification model with multiple outcomes, the output may look like a matrix where the number of rows is the number of models and the columns represent the different outcomes.

In [2]:
# 3 models with 5 outcomes:
y = softmax(np.random.random((3,5)))
print(y)

# All probabilities add up to 1
assert np.allclose(y.sum(axis=1), [1.,1.,1.])

[[ 0.2756  0.1532  0.1844  0.1562  0.2306]
 [ 0.1298  0.2145  0.1948  0.2297  0.2312]
 [ 0.1725  0.1606  0.2515  0.2531  0.1624]]


Actually, it's really simple to take the ensemble average.

In [3]:
y_bar = softmax(np.log(y).mean(axis=0))
print(y_bar)

assert np.isclose(y_bar.sum(), 1.)

# It is not the same as taking the mean directly.
print(y.mean(axis=0))

[ 0.1872  0.1777  0.2126  0.2129  0.2096]
[ 0.1926  0.1761  0.2102  0.213   0.2081]


Why, intuitivly, the ensemble mean is better than just taking the mean? I will try to make one model disagree heavily with the others. Model number zero gives the opposite results as the other two models, which agree perfectly.

In [4]:
y[0,0] = 0.
y[0,1:] = 1.
y[1:,0] = 1.
y[1:,1:] = 0.
# Can't let the probability be zero because of the log
y.clip(min = 1.e-10, out = y)
print(y)

[[  1.0000e-10   1.0000e+00   1.0000e+00   1.0000e+00   1.0000e+00]
 [  1.0000e+00   1.0000e-10   1.0000e-10   1.0000e-10   1.0000e-10]
 [  1.0000e+00   1.0000e-10   1.0000e-10   1.0000e-10   1.0000e-10]]


Now compare the outcomes using the two types of averaging:

In [5]:
print(softmax(np.log(y).mean(axis=0)))
print(y.mean(axis=0))

[  9.9815e-01   4.6330e-04   4.6330e-04   4.6330e-04   4.6330e-04]
[ 0.6667  0.3333  0.3333  0.3333  0.3333]


So this method appears to ignore outliers!